In [ ]:
from __future__ import division
import string
import os
import numpy as np
import caffe
import surgery
import os
import sys
import caffe
from datetime import datetime
from PIL import Image
import six.moves.cPickle as pickle
from operator import itemgetter, attrgetter
import scipy.io as sio

GPU = 1 # Set GPU Device 
target = "test"
models = ['fcn8s','fcn16s', 'fcn32s' ,'fcnResNet152', 'fcnResNet101','fcnResNet50'] # Set Models Name, should be same as folder name

TST_TAG_LIST = ["overall accuracy", "mean accuracy", "mean IU", "fwavacc", "per-class IU", 'loss']

In [ ]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

def compute_hist(scores, save_dir, dataset, layer='score', gt='label'):
    n_cl = scores.shape[1]
    if save_dir:
        os.mkdir(save_dir)
    hist = np.zeros((n_cl, n_cl))
    for idx,name in enumerate(dataset):
        hist += fast_hist(gt[idx].flatten(), scores[idx].argmax(0).flatten(), n_cl)
        
        if save_dir:
            im = Image.fromarray(scores[idx].argmax(0).astype(np.uint8), mode='P')
            im.save(os.path.join(save_dir, name + '.png'))

    return hist


def get_score(solver, dataset, layer='score', gt='label'):
    solver.test_nets[0].share_with(solver.net)
    net = solver.test_nets[0]
    n_cl = net.blobs[layer].channels
    score = np.zeros((len(dataset), n_cl, 256, 256))
    label = np.zeros((len(dataset), 256, 256))
    for idx,_ in enumerate(dataset):
        net.forward()
        score[idx] = net.blobs[layer].data[0]
        label[idx] = net.blobs[gt].data[0, 0]
    return score, label


def do_seg_tests(scores, save_format, dataset, layer='score', gt='label'):
    ret = {x:-1 for x in TST_TAG_LIST}
    if save_format:
        save_format = save_format.format(2500)
    iter = 1
    hist = compute_hist(scores, save_format, dataset, layer, gt)
    # overall accuracy
    acc = np.diag(hist).sum() / hist.sum()
    print '>>>', datetime.now(), 'Iteration', iter, 'overall accuracy', acc
    ret['overall accuracy'] = acc
    # per-class accuracy
    acc = np.diag(hist) / hist.sum(1)
    print '>>>', datetime.now(), 'Iteration', iter, 'mean accuracy', np.nanmean(acc)
    ret['mean accuracy'] = np.nanmean(acc)
    #per-class IU
    iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
    print '>>>', datetime.now(), 'Iteration', iter, 'mean IU', np.nanmean(iu)
    ret['mean IU'] = np.nanmean(iu)
    #IU per class
    print 'Iteration', iter, 'per-class IU', iu
    ret['per-class IU'] = iu
    freq = hist.sum(1) / hist.sum()
    print '>>>', datetime.now(), 'Iteration', iter, 'fwavacc', (freq[freq > 0] * iu[freq > 0]).sum()
    ret['fwavacc'] = (freq[freq > 0] * iu[freq > 0]).sum()
    return ret
    
def setup(model):
    with open('solver.prototxt') as f:
        with open('temp','w') as w:
            w.write('train_net: \''+model+'-trainval.prototxt\'\n')
            w.write('test_net: \'' +model+ '-'+target+'.prototxt\'\n')
            flag = True
            for line in f:
                if flag:
                    next(f)
                    flag = False
                    continue
                w.write(line)

    os.system('mv temp solver.prototxt')
    
def load_class_names():
    sem_names = []
    with open("../data/sift-flow/classes.txt","r") as f:
        for _ in xrange(4):
            next(f)
        for _ in xrange(1,34):
            sem_names.append(next(f).split(' ')[1][:-1])
    return sem_names

def write_PerClassIU(IU_list, model):
    sem_names = load_class_names()
    with open("Per-ClassIU.txt","a") as f:
        f.write("===============\n"+model+"\n")
        lst = []
        for i, name in enumerate(sem_names):
            lst.append( (name, IU_list[i]) ) 
        for line in sorted(lst, key = itemgetter(1) , reverse=True):
            if  not (line[0] in ["cow", "desert", "moon"] ):
                f.write(line[0] + " ")
                f.write("%2.1f\n" % (line[1] * 100) )
                

In [ ]:
try:
    import setproctitle
    setproctitle.setproctitle(os.path.basename(os.getcwd()))
except:
    pass

# init
caffe.set_device(GPU)
caffe.set_mode_gpu()
# scoring
test = np.loadtxt('../data/sift-flow/'+target+'.txt', dtype=str)

scores_sem = np.zeros((len(test), 33, 256, 256))
scores_geo = np.zeros((len(test), 3, 256, 256))

for model in models:
    print datetime.now(), "Running model: ", model
    model_path = '../siftflow-' + model + '/'
    setup(model)
    weights = model_path + '_iter_2500'+'.caffemodel'
    solver = caffe.SGDSolver('solver.prototxt')
    solver.net.copy_from(weights)
    # surgeries
    interp_layers = [k for k in solver.net.params.keys() if 'up' in k]
    surgery.interp(solver.net, interp_layers)
    
    sem, gt_sem = get_score(solver, test, layer='score_sem', gt='sem')
    geo, gt_geo = get_score(solver, test, layer='score_geo', gt='geo')
    
    scores_sem += sem
    scores_geo += geo

scores_sem = scores_sem / (1.0 * len(models))
scores_geo = scores_geo / (1.0 * len(models))    
print "Final test:"
seg_result = do_seg_tests(scores_sem, False, test, layer='score_sem', gt=gt_sem)
geo_result = do_seg_tests(scores_geo, False, test, layer='score_geo', gt=gt_geo)


with open('Ensemble-'+target+'-sem.pkl', 'wb') as f:
    pickle.dump(seg_result, f)
with open('Ensemble-'+target+'-geo.pkl', 'wb') as f:
    pickle.dump(geo_result, f)


In [ ]:
#write_PerClassIU(seg_result["per-class IU"], "Ensemble")
for tag in TST_TAG_LIST:
        if tag != "loss" and tag != "per-class IU":
            print tag + ": %2.1f" % (seg_result[tag]*100)
print "geom. acc. : %2.1f" % (geo_result["overall accuracy"]*100)